# Video Stream MobileNet_Tuned_Hyperband2 e Document Info

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os

In [2]:
from pytesseract import Output
from codicefiscale import codicefiscale
from datetime import datetime
import pytesseract
import argparse
import cv2

## Definisco la funzione che estrae le informazioni dall'immagine del retro della tessera sanitaria

In [3]:
def get_document_info(image):
    
    #Leggo l'immagine
    #image = cv2.imread(r'C:\Users\matti\Desktop\cf_retro_simo.jpg')
    #rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    #Dall'immagine alle informazioni del documento
    results = pytesseract.image_to_data(image, output_type=Output.DICT)
    
    testo=results['text']

    lista=[]
    for parola in testo:
        if len(parola) >=4 and len(parola) <= 16 and " " not in parola:
            lista.append(parola)
            
    #Estraggo data e ora dell'identificazione
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    
    return lista, dt_string

## Importo il Face Detector e il Face_Mask Detector

In [4]:
prototxtPath = os.path.sep.join(['C:/Users/matti/OneDrive - Universita degli Studi di Milano-Bicocca/Uni/LM-DataScience/1-Anno/Data Science Lab/Progetto_Face_Mask/face-mask-detector/face_detector', "deploy.prototxt"])
weightsPath = os.path.sep.join(['C:/Users/matti/OneDrive - Universita degli Studi di Milano-Bicocca/Uni/LM-DataScience/1-Anno/Data Science Lab/Progetto_Face_Mask/face-mask-detector/face_detector',"res10_300x300_ssd_iter_140000.caffemodel"])
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
maskNet = load_model('C:/Users/matti/OneDrive - Universita degli Studi di Milano-Bicocca/Uni/LM-DataScience/1-Anno/Data Science Lab/Progetto_Face_Mask/Modelli_Trained/MobileNet_Tuned_Hyperband2')

## Test Webcam

In [5]:
#con src=0 si utilizza la webcam del pc
vs = VideoStream(src=0).start()
time.sleep(2.0)

Definisco una funzione che prende in input un frame video, il modello per identificare il volto di una persona (FaceNet) e il modello che permette di stabilire se è presente un volto con e senza maschera (MaskNet).

In [6]:
def detect_and_predict_mask(frame, faceNet, maskNet):

    #Estraggo i valori di hight e width  
    (h, w) = frame.shape[:2]
    #Costruisco un blob dall'immagine
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
        (104.0, 177.0, 123.0))
    #Passo il blob al Face Network che mi fornisce le face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    #Inizializzo una lista per le facce, le relative posizioni 
    #e le previsioni del mask detector
    faces = []
    locs = []
    preds = []
    
    for i in range(0, detections.shape[2]):
        #Per ciascuna detections, estraggo il valore della confidenza   
        confidence = detections[0, 0, i, 2]
        #Considero solo le detections che hanno una condifenza maggiore di una determinata soglia
        if confidence > 0.5:
            #Considero le coordinate del box in cui è presente la faccia
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            #Mi assicuro che il box in cui è presente l'immagine sta all'interno del frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            
            #Estraggo dall'immagine la ROI
            face = frame[startY:endY, startX:endX]
            #Converto l'immagine da BGR a RGB
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            #Resize della faccia in quanto la rete LeNet è allenata su immagini di dimensione 224x224
            face = cv2.resize(face, (224, 224))
            #Trasformo in array
            face = img_to_array(face)
            #Preprocess dell'immagine
            face = preprocess_input(face)
            #Aggiungo la faccia alla lista di facce
            faces.append(face)
            #Aggiungo la posizione della faccia alla lista delle posizioni
            locs.append((startX, startY, endX, endY))
    
    #Se ho trovato almeno una faccia:
    if len(faces) > 0:
        #Converto la faccia in un array di numpy
        faces = np.array(faces, dtype="float32")
        #Passo la faccia tramite la rete allenata per individuare la mascherina
        preds = maskNet.predict(faces, batch_size=32)
    
    #Viene restituita una tupla contenente la posizione della faccia e
    #le probabilità associate alle labels mask/no mask
    return (locs, preds)

In [7]:
while True:
    #Continuo a cicliare sui frame video
    frame = vs.read()
    #Formatto il frame
    frame = imutils.resize(frame, width=400)
        
    #Tramite la funzione definita sopra, determino se nel frame sono presenti
    #delle facce e determino se indossano o meno la maschera
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    
    #Ripeto questa operazione per ogni faccia individuata:
    for (box, pred) in zip(locs, preds):
        
        #Estraggo il box e le predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred
        
        #Determino se la faccia ha o meno la maschera e definisco
        #il colore rosso per No Mask e verde per Mask. 
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        
        #Formatto la probabilità delle label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
       
        #Predispongo la visualzzione del frame, della label
        # e della probabilità prevista dal modello
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        
    #Mostro il frame con il risultato della previsione
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
        
    if label == "Mask":
        key=ord("q")
    
    #Lo streaming viene stoppato con q.
    if key == ord("q"):
        break
    
    
time.sleep(5)
     
cam = cv2.VideoCapture(0)
cv2.namedWindow("test")
ret, frame = cam.read()
img_name = "document.jpg"
cv2.imwrite(img_name, frame)
print("{} written!".format(img_name))
cam.release()
cv2.destroyAllWindows()
    
get_document_info('document.jpg')
            
cv2.destroyAllWindows()
vs.stop()

document.jpg written!


TesseractError: (1, 'Tesseract Open Source OCR Engine v5.0.0-alpha.20200328 with Leptonica Error, cannot read input file C:\\Users\\matti\\OneDrive - Universita degli Studi di Milano-Bicocca\\Uni\\LM-DataScience\\1-Anno\\Data Science Lab\\opencv_frame_0.jpg: No such file or directory Error during processing.')

In [6]:
vs.stop()

NameError: name 'vs' is not defined

In [11]:
import cv2
cam = cv2.VideoCapture(0)
cv2.namedWindow("test")
img_counter = 0
while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)
    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "opencv_frame_{}.jpg".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1
cam.release()
cv2.destroyAllWindows()

opencv_frame_0.jpg written!
Escape hit, closing...
